In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png 


# zad1
dom = cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(dom,'gray')
plt.show()
size=5
blur = cv2.GaussianBlur(dom, (size,size), 15)
LoG = cv2.Laplacian(blur, cv2.CV_32F)



thr=120


def crossing(LoG, thr):
    (h,w) = LoG.shape
    finalImage = np.zeros([h,w])
    for y in range(1, h-1):
        for x in range(1, w-1):
            surroundings = LoG[y-1:y,x-1:x]
            mini = np.min(surroundings)
            maxi = np.max(surroundings)      
            if (mini*maxi<0):
                if (LoG[y,x]<=0):
                    finalImage[y,x] = abs(LoG[y,x]) + maxi
                else:
                    finalImage[y,x] = LoG[y,x] + abs(mini)
            else:
                finalImage[y,x]=LoG[y,x]
    nI = cv2.normalize(finalImage,None,0,255,cv2.NORM_MINMAX)
    nI = (nI>=thr)*255
    finalImage = cv2.medianBlur(nI.astype('uint8'), 3)
    return finalImage

crs  = crossing(LoG,thr)

plt.imshow(crs,'gray')
plt.show()





def _canny(img):
    blur = cv2.GaussianBlur(img, (5, 5), 1.4) 
       
  
    gx = cv2.Sobel(np.float32(blur), cv2.CV_64F, 1, 0, 3) 
    gy = cv2.Sobel(np.float32(blur), cv2.CV_64F, 0, 1, 3) 
      

    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees = True) 
    mag_max = np.max(mag) 
    weak = mag_max * 0.1
    strong = mag_max * 0.5

    h, w = img.shape 
      
    for x in range(w): 
        for y in range(h): 
               
            grad = ang[y, x] 
            grad = abs(grad-180) if abs(grad)>180 else abs(grad) 
               
           
            if grad<= 22.5: 
                neighb_1_x, neighb_1_y = x-1, y 
                neighb_2_x, neighb_2_y = x + 1, y 
              
          
            elif grad>22.5 and grad<=(22.5 + 45): 
                neighb_1_x, neighb_1_y = x-1, y-1
                neighb_2_x, neighb_2_y = x + 1, y + 1
              
          
            elif grad>(22.5 + 45) and grad<=(22.5 + 90): 
                neighb_1_x, neighb_1_y = x, y-1
                neighb_2_x, neighb_2_y = x, y + 1
              
            
            elif grad>(22.5 + 90) and grad<=(22.5 + 135): 
                neighb_1_x, neighb_1_y = x-1, y + 1
                neighb_2_x, neighb_2_y = x + 1, y-1
              
            
            elif grad>(22.5 + 135) and grad<=(22.5 + 180): 
                neighb_1_x, neighb_1_y = x-1, y 
                neighb_2_x, neighb_2_y = x + 1, y 
               
            
            if w>neighb_1_x>= 0 and h>neighb_1_y>= 0: 
                if mag[y, x]<mag[neighb_1_y, neighb_1_x]: 
                    mag[y, x]= 0
                    continue
   
            if w>neighb_2_x>= 0 and h>neighb_2_y>= 0: 
                if mag[y, x]<mag[neighb_2_y, neighb_2_x]: 
                    mag[y, x]= 0
   
    weak_ids = np.zeros_like(img) 
    strong_ids = np.zeros_like(img)               
    ids = np.zeros_like(img) 
       
    
    for x in range(w): 
        for y in range(h): 
              
            grad_mag = mag[y, x] 
              
            if grad_mag<weak: 
                mag[y, x]= 0
            elif strong>grad_mag>= weak: 
                ids[y, x]= 1
            else: 
                ids[y, x]= 2
       
       
   
    return mag 


plt.imshow(_canny(dom),'gray')
plt.show()





# zad2

edges = cv2.Canny(dom,100,200,None,3,1)
plt.imshow(edges,'gray')
plt.show()

